In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import glob
import re
pd.set_option('display.max_columns', None)

In [2]:
# geostl nghbd codes:
    # gravois park = 19
    # benton park west = 30
    # dutchtown = 16
    # mount pleasant = 17
# https://dynamic.stlouis-mo.gov/citydata/newdesign/sqlsearch.cfm
# gravois_total_parcels = 1733
# benton_park_west_total_parcels = 1693
# dutchtown_total_parcels = 4850

In [3]:
csv_folder = 'stl_vacancy_data/'
path = "stl_vacancy_data/*.csv"
csv_list = []

gj_vacancy_cat_df = pd.DataFrame(columns=['Date', 'V_Indeterminate', 'V_Possible', 'V_Very_likely', 'V_Definite'])
gj_burden_cat_df = pd.DataFrame(columns=['Date', 'B_Zero', 'B_Low', 'B_Medium', 'B_High'])

for fname in glob.glob(path):
    csv_name = re.findall(r'stl_vacancy_data_\d\d\d\d-\d\d-\d\d.csv', fname)[0]
    csv_list.append(csv_name)

In [ ]:
parcels = pd.read_csv('grav-jeff-parcels.csv')
nhd_num_list = [16, 17, 19, 30]
mask = parcels['NBRHD'].isnin(nhd_num_list)
parcels_df = parcels[mask]

parcels_df['SITEADDR'] = parcels_df['SITEADDR'].replace(r'\s+', ' ', regex=True)

In [ ]:
gj_vacancy_cat_df = pd.DataFrame(columns=['Date', 'V_Indeterminate', 'V_Possible', 'V_Very_likely', 'V_Definite'])
gj_burden_cat_df = pd.DataFrame(columns=['Date', 'B_Zero', 'B_Low', 'B_Medium', 'B_High'])

In [ ]:
def combine_columns(row):
    return str(row['StAddrNum']) + ' ' + row['StNameFull']

In [ ]:
class VacancyTransformer():
    def __init__(self, csv_name):
        self.date = re.findall(r'\d\d\d\d-\d\d-\d\d', csv_name)[0]
        self.csv_name = csv_name
        
    def load_raw_df(self):
        self.raw_df = pd.read_csv(csv_folder+self.csv_name)
        return self.raw_df

    def create_regional_df(self, parcel_df):

        # nhd_names = ['Gravois Park', 'Benton Park West', 'Dutchtown', 'Mount Pleasant']
        # mask = self.raw_df['NhdName'].isin(nhd_names)
        # regional_df = raw_df[mask]

        full_nhds = ['Gravois Park', 'Benton Park West']
        partial_nhds = ['Dutchtown', 'Mount Pleasant']

        full_mask = self.raw_df['NhdName'].isin(full_nhds)
        bpw_gp_df = raw_df[full_mask]

        partial_mask = self.raw_df['NhdName'].isin(partial_nhds)
        partial_df = raw_df[partial mask]

        matching_parcels = []
        for handle in list(partial_nhds['Handle']): #sdlkfjsdlkfjslkdjfslkdjflskdjflksdjflskdjflksdjflsksdjflksdjf
            if handle in list(parcels_df['HANDLE']):
                matching_parcels.append(handle)

        handle_mask = partial_nhds['NhdName'].isin(matching_parcels)
        handle_match_df = partial_nhds[handle_mask]

        partial_df['SITEADDR'] = partial_df.apply(combine_columns, axis=1)
        
        matching_parcels = []
        for site_addr in list(partial_df['SITEADDR']):
            if site_addr in list(parcels_df['SITEADDR']):
                matching_parcels.append(site_addr)

        addr_mask = partial_nhds['SITEADDR'].isin(matching_parcels)
        addr_match_df = partial_nhds[addr_mask]

        joint_df = pd.concat([handle_match_df, addr_match_df], ignore_index=True)
        regional_df = joint_df.drop_duplicates(ignore_index=True)
        
        # gj_addr = list(regional_df['SITEADDR'])
        # site_mask = parcels['SITEADDR'].isin(gj_addr)
        # site_handles = parcels['HANDLE'][site_mask]

        # for handle in list(regional_df['Handle']):
        #     if handle in list(parcels_df['HANDLE']):
        #         matching_parcels.append(handle)
        #     elif 

        return regional_df
        
    def calc_vacancy_cats(self, df, aggregate_df):
        counts = df['VacancyCat'].value_counts()
        data = [self.date] + counts[['Indeterminant', 'Possible', 'Very Likely', 'Definite']].tolist()
        
        aggregate_df.loc[len(aggregate_df.index)] = data

    def calc_burden_cats(self, df, aggregate_df):
        counts = df['BurdenCat'].value_counts()
        zero_cat = counts['Zero']
        low_cat = sum(counts[['Minimal', 'Very Low', 'Low']])
        med_cat = sum(counts[['Medium Low', 'Medium', 'Medium High', 'Somewhat High']])
        high_cat = sum(counts[['High', 'Very High', 'Extremely High']])
        
        aggregate_df.loc[len(aggregate_df.index)] = [self.date, zero_cat, low_cat, med_cat, high_cat]

    def calc_groupby_counts(self, df, cat_list):
        
        # valid = {'stl','bpw-gp', 'dutchtown'}
        # if region not in valid:
        #     raise ValueError("results: status must be one of %r." % valid)
        
        for cat in cat_list:
            
            # VacancyCat:
            vac_df = df.groupby(cat)['VacancyCat'].value_counts().to_frame().unstack()
            vac_df.columns = vac_df.columns.droplevel()
            vac_df.fillna(value=0, inplace=True)
            vac_value_name = vac_df.columns.name
            vac_index_name = vac_df.index.name
            vac_df = vac_df[['Indeterminant', 'Possible', 'Very Likely', 'Definite']]
            vac_df.rename(columns={'Indeterminant': 'V_Indeterminant',
                               'Possible': 'V_Possible',
                               'Very Likely': 'V_Very_Likely',
                               'Definite': 'V_Definite'},
                          inplace=True)
            vac_df.reset_index(inplace=True)
            vac_df.insert(loc=0, column='Date', value=self.date)
            vac_df.to_csv(f'data/temp/type_vacancy_data/{region}/{region}_{vac_index_name}_{vac_value_name}_{self.date}.csv')

            # BurdenCat
            bur_df =  df.groupby(cat)['BurdenCat'].value_counts().to_frame().unstack()
            bur_df.columns = bur_df.columns.droplevel()
            bur_df.fillna(value=0, inplace=True)
            bur_index_name = bur_df.index.name
            bur_value_name = bur_df.columns.name
            bur_df['B_Zero'] = bur_df['Zero']
            bur_df['B_Low'] = bur_df['Minimal'] + bur_df['Very Low'] + bur_df['Low']
            bur_df['B_Medium'] = bur_df['Medium Low'] + bur_df['Medium'] + bur_df['Medium High'] + bur_df['Somewhat High']
            bur_df['B_High'] = bur_df['High'] + bur_df['Very High'] + bur_df['Extremely High']
            bur_df.reset_index(inplace=True)
            bur_df.insert(loc=0, column='Date', value=self.date)
            bur_df = bur_df[['Date', 'Type', 'B_Zero', 'B_Low', 'B_Medium', 'B_High']]
            bur_df.to_csv(f'data/temp/type_burden_data/{region}/{region}_{bur_index_name}_{bur_value_name}_{self.date}.csv')